In [1]:
!pip install -q streamlit pyngrok


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 52.4 MB/s eta 0:00:00


In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token("YOUR TOKEN")


In [12]:
!pip install langchain-groq tavily-python langgraph PyPDF2 reportlab sentence-transformers langsmith


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 11.1 MB/s eta 0:00:00


In [ ]:


%%writefile backend.py

import os
os.environ["GROQ_API_KEY"] = "YOUR TOKEN"
os.environ["TAVILY_API_KEY"] = "YOUR TOKEN"
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = "YOUR TOKEN"
import os, re, json, datetime, html, time, hashlib
from dataclasses import dataclass
from typing import List, Optional, TypedDict
from pathlib import Path
import importlib

from functools import lru_cache
from langgraph.graph import StateGraph, END
from IPython.display import Markdown, display
os.environ['EMBEDDING_DEBUG'] = '0'

# Global buffer for ALL raw LLM logs

GLOBAL_RAW_LOG: List[str] = []

#  LangSmith tracing

try:
    from langsmith import traceable
except ImportError:
    # No-op decorator if langsmith not installed
    def traceable(*targs, **tkwargs):
        def decorator(fn):
            return fn
        return decorator

# API Keys (Groq + Tavily)

GROQ_API_KEY = os.getenv("GROQ_API_KEY", "").strip()
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY", "").strip()

if not GROQ_API_KEY or not TAVILY_API_KEY:
    raise SystemExit(
        "ERROR: GROQ_API_KEY and TAVILY_API_KEY must be set in environment.\n"
        "In Colab, run:\n"
        "import os\n"
        "os.environ['GROQ_API_KEY'] = 'your_groq_key'\n"
        "os.environ['TAVILY_API_KEY'] = 'your_tavily_key'"
    )


# External clients & PDF lib (reading)

try:
    from tavily import TavilyClient
except Exception:
    raise RuntimeError("Missing tavily library. Install with: pip install tavily-python")

# PDF lib for reading user PDFs
_pdf_lib = None
if importlib.util.find_spec("PyPDF2"):
    import PyPDF2 as _pdf_lib
else:
    _pdf_lib = None  # PDF ingestion will error if not installed

tavily_client = TavilyClient(api_key=TAVILY_API_KEY)


# PDF writer for saving raw LLM outputs

try:
    from reportlab.pdfgen import canvas
    from reportlab.lib.pagesizes import letter
    _pdf_writer_available = True
except ImportError:
    _pdf_writer_available = False


def save_raw_to_pdf(text: str, filename: str):
    if not _pdf_writer_available:
        print("[PDF] reportlab not installed; skipping PDF save.")
        return
    try:
        c = canvas.Canvas(filename, pagesize=letter)
        width, height = letter
        x_margin = 40
        y = height - 50
        max_width_chars = 110
        for line in text.split("\n"):
            while len(line) > max_width_chars:
                chunk = line[:max_width_chars]
                line = line[max_width_chars:]
                if y < 40:
                    c.showPage()
                    y = height - 50
                c.drawString(x_margin, y, chunk)
                y -= 15
            if y < 40:
                c.showPage()
                y = height - 50
            c.drawString(x_margin, y, line)
            y -= 15
        c.save()
        print(f"[PDF Saved] {filename}")
    except Exception as e:
        print(f"[PDF ERROR] Failed to save PDF '{filename}': {e}")


def save_all_raw_to_one_pdf(filename: str = "all_llm_raw_output.pdf"):
    if not GLOBAL_RAW_LOG:
        print("[PDF] No raw LLM data to save.")
        return
    full_text = "\n".join(GLOBAL_RAW_LOG)
    save_raw_to_pdf(full_text, filename)



# Groq LLM via LangChain

from langchain_groq import ChatGroq

GROQ_MODEL = os.getenv("GROQ_MODEL", "llama-3.1-8b-instant")
_LLM_CACHE_PATH = Path("/tmp/groq_inference_cache.json")
try:
    _LLM_CACHE = json.load(open(_LLM_CACHE_PATH)) if _LLM_CACHE_PATH.exists() else {}
except Exception:
    _LLM_CACHE = {}


def _save_llm_cache():
    try:
        json.dump(_LLM_CACHE, open(_LLM_CACHE_PATH, "w"), indent=2, ensure_ascii=False)
    except Exception:
        pass


def _prompt_key(prompt: str) -> str:
    return hashlib.sha256(prompt.encode("utf-8")).hexdigest()


groq_llm = ChatGroq(
    model=GROQ_MODEL,
    temperature=0.3,
    max_tokens=800,
    groq_api_key=GROQ_API_KEY,
)


@traceable(name="groq_llm_call")
def groq_chat(
    prompt: str,
    *,
    use_cache: bool = True,
    max_retries: int = 3,
    max_new_tokens: int = 800,
    timeout: int = 60,
) -> str:

    key = _prompt_key(prompt)

    # ✅ Optional cache usage
    if use_cache and key in _LLM_CACHE:
        txt = _LLM_CACHE[key]
        GLOBAL_RAW_LOG.append(
            f"\n\n=== CACHED LLM CALL ===\n"
            f"Timestamp: {datetime.datetime.now(datetime.timezone.utc).isoformat()}\n"
            f"Prompt:\n{prompt}\n\nResponse:\n{txt}\n"
        )
        return txt

    # ---- real LLM call ----
    try:
        resp = groq_llm.invoke(prompt)
    except Exception as e:
        print(f"Groq LLM error: {e}")
        return ""

    if hasattr(resp, "content"):
        if isinstance(resp.content, str):
            txt = resp.content
        elif isinstance(resp.content, list):
            txt = "".join(str(p) for p in resp.content)
        else:
            txt = str(resp.content)
    else:
        txt = str(resp)

    txt = (txt or "").strip()

    # ✅ Save only if cache is enabled
    if use_cache:
        _LLM_CACHE[key] = txt
        _save_llm_cache()

    GLOBAL_RAW_LOG.append(
        f"\n\n=== RAW LLM CALL ===\n"
        f"Timestamp: {datetime.datetime.now(datetime.timezone.utc).isoformat()}\n"
        f"Prompt:\n{prompt}\n\nResponse:\n{txt}\n"
    )

    return txt

@lru_cache(maxsize=4096)
def cached_grade(prompt: str) -> int:
    raw = groq_chat(prompt)
    try:
        return int(json.loads(re.search(r"\{.*\}", raw).group())["score"])
    except Exception:
        return 0



# Checkpoint structure

@dataclass
class Checkpoint:
    id: str
    topic: str
    objectives: List[str]
    success_criteria: str

CHECKPOINTS = [

    # ==================================================
    # CP1 — Neural Network Basics
    # ==================================================
    Checkpoint(
        id="cp1",
        topic="Basics of Neural Networks",
        objectives=[
            "Explain how an artificial neuron combines inputs using weights and a bias",
            "Describe the roles of input layers hidden layers and output layers",
            "Explain how data moves forward through a neural network during prediction",
        ],
        success_criteria=(
            "Learner can verbally explain how a simple feedforward neural network "
            "processes inputs to produce an output."
        ),
    ),

    # ==================================================
    # CP2 — Loss Functions
    # ==================================================
    Checkpoint(
        id="cp2",
        topic="Loss Functions",
        objectives=[
            "Explain why a loss function is needed to measure prediction error",
            "Describe the difference between low loss and high loss",
            "Identify a suitable loss function for a simple regression or classification task",
        ],
        success_criteria=(
            "Learner can explain how loss functions quantify model error "
            "and why minimizing loss is important."
        ),
    ),

    # ==================================================
    # CP3 — Gradient Descent
    # ==================================================
    Checkpoint(
        id="cp3",
        topic="Gradient Descent",
        objectives=[
            "Explain why minimizing a loss function improves model performance",
            "Describe the gradient as the direction and size of the steepest loss change",
            "Explain how weights are updated step by step using the gradient",
        ],
        success_criteria=(
            "Learner can explain how gradient descent reduces error "
            "through repeated weight updates."
        ),
    ),

    # ==================================================
    # CP4 — Learning Rate
    # ==================================================
    Checkpoint(
        id="cp4",
        topic="Learning Rate",
        objectives=[
            "Explain what the learning rate controls during training",
            "Describe what happens when the learning rate is too high or too low",
            "Explain why choosing an appropriate learning rate matters",
        ],
        success_criteria=(
            "Learner can explain the effect of learning rate "
            "on training stability and convergence speed."
        ),
    ),

    # ==================================================
    # CP5 — Activation Functions
    # ==================================================
    Checkpoint(
        id="cp5",
        topic="Activation Functions",
        objectives=[
            "Explain why activation functions are needed in neural networks",
            "Describe the behavior of sigmoid tanh and relu functions",
            "Choose an activation function for a simple task and justify the choice",
        ],
        success_criteria=(
            "Learner can select and justify an activation function "
            "for a given problem."
        ),
    ),

    # ==================================================
    # CP6 — Backpropagation
    # ==================================================
    Checkpoint(
        id="cp6",
        topic="Backpropagation",
        objectives=[
            "Explain how the chain rule connects gradients across network layers",
            "Describe how prediction error flows backward from output to hidden layers",
            "Explain why backpropagation enables efficient weight updates",
        ],
        success_criteria=(
            "Learner can explain backpropagation intuitively without equations "
            "using a simple neural network example."
        ),
    ),

    # ==================================================
    # CP7 — Overfitting and Generalization
    # ==================================================
    Checkpoint(
        id="cp7",
        topic="Overfitting and Generalization",
        objectives=[
            "Explain what overfitting means in machine learning",
            "Describe the difference between training performance and test performance",
            "Identify common signs that a model is overfitting",
        ],
        success_criteria=(
            "Learner can explain why a model that performs well on training data "
            "may fail on unseen data."
        ),
    ),

    # ==================================================
    # CP8 — Train Validation and Test Data
    # ==================================================
    Checkpoint(
        id="cp8",
        topic="Train Validation and Test Data",
        objectives=[
            "Explain why data is split into training validation and test sets",
            "Describe the role of each dataset during model development",
            "Explain how validation data helps improve model decisions",
        ],
        success_criteria=(
            "Learner can explain the purpose of each data split "
            "and how they prevent misleading performance evaluation."
        ),
    ),

    # ==================================================
    # CP9 — Weight Initialization
    # ==================================================
    Checkpoint(
        id="cp9",
        topic="Weight Initialization",
        objectives=[
            "Explain why initial weights affect neural network training",
            "Describe problems caused by very large or very small initial weights",
            "Explain how good initialization helps gradients flow during training",
        ],
        success_criteria=(
            "Learner can explain why proper weight initialization "
            "improves training stability and convergence."
        ),
    ),

    # ==================================================
    # CP10 — Regularization Techniques
    # ==================================================
    Checkpoint(
        id="cp10",
        topic="Regularization Techniques",
        objectives=[
            "Explain why regularization is used to reduce overfitting",
            "Describe how L1 L2 and dropout regularization affect model training",
            "Choose a regularization technique for a simple scenario and justify it",
        ],
        success_criteria=(
            "Learner can explain how regularization improves generalization "
            "and select an appropriate technique for a given problem."
        ),
    ),
]



# Agent State

class AgentState(TypedDict):
    cp_id: str
    checkpoint: Optional[Checkpoint]
    user_notes: str
    user_pdfs: List[str]
    gathered_context: str
    context_sources: List[str]
    relevance_score_model: Optional[int]
    refetch_attempted: bool
    score_meta: Optional[dict]

    processed_chunks: List[str]
    questions: List[str]
    learner_answers: List[str]

    score_percent: Optional[float]
    pass_threshold_met: Optional[bool]

    question_scores: Optional[List[dict]]   # ✅ ADD THIS

    temp_vector_store: Optional[dict]
    feynman_explanation: Optional[str]
    feynman_rounds: int
    focus_concepts: Optional[List[str]]
    failed_objectives: List[str]


_score_re = re.compile(r"\b([1-5])\b")


def parse_score_from_text(raw: str) -> int:
    if not raw:
        return 3
    m = _score_re.search(raw)
    return int(m.group(1)) if m else 3



# PDF extraction helpers

def extract_text_from_pdf(path: str) -> str:
    if not _pdf_lib:
        raise RuntimeError("PyPDF2 not installed. `pip install PyPDF2` to enable PDF ingestion.")
    text = []
    with open(path, "rb") as f:
        reader = _pdf_lib.PdfReader(f)
        for p in reader.pages:
            try:
                page_text = p.extract_text() or ""
            except Exception:
                page_text = ""
            if page_text:
                text.append(page_text)
    return "\n".join(text)


def gather_texts_from_pdfs(paths: List[str]) -> str:
    out = ""
    for p in paths:
        try:
            t = extract_text_from_pdf(p)
            if t.strip():
                out += f"\n--- PDF: {os.path.basename(p)} ---\n{t}\n"
        except Exception as e:
            print(f"Failed to read PDF {p}: {e}")
    return out

def expand_text(text: str, cp: Checkpoint) -> str:
    if not text.strip():
        return text

    prompt = f"""
You are a technical instructor.

Write a DETAILED and STRUCTURED explanation strictly for learning purposes.

TOPIC:
{cp.topic}

LEARNING OBJECTIVES:
{json.dumps(cp.objectives, ensure_ascii=False)}

STYLE RULES:
- Be factual and instructional
- Use clear headings for each objective
- Explain concepts step by step
- Use simple technical examples only
- Avoid metaphors stories or creative analogies
- Avoid unnecessary verbosity
- Do NOT oversimplify
- Do NOT be conversational
- Write like a concise textbook or lecture note

DEPTH REQUIREMENTS:
- Each objective must have at least 2–3 clear paragraphs
- Explain key terms explicitly
- Maintain logical flow
- Target ~700–1000 words total

SOURCE MATERIAL:
\"\"\"{text[:8000]}\"\"\"

Write the explanation:
"""
    return groq_chat(prompt, use_cache=False).strip()



# Summarizer (uses Groq)

def summarize_text(text: str) -> str:
    if not text.strip():
        return text
    prompt = f"""
Summarize the following text into a focused explanation matching the learning objectives.
Keep it concise, clean, and relevant.

Text:
\"\"\"{text[:5000]}\"\"\"\n
Summary:
"""
    summary = groq_chat(prompt)
    return summary.strip() or text[:5000]



# Tavily wrapper with rate limiting (< 10 searches/min)

_search_timestamps: List[float] = []


def _enforce_search_rate_limit():
    global _search_timestamps
    now = time.time()
    _search_timestamps = [t for t in _search_timestamps if now - t < 60]
    if len(_search_timestamps) >= 9:
        oldest = min(_search_timestamps)
        wait = 60 - (now - oldest)
        if wait > 0:
            print(f"[Rate Limit] Tavily search rate reached. Waiting {wait:.1f}s to stay under 10/min...")
            time.sleep(wait)
        now = time.time()
        _search_timestamps = [t for t in _search_timestamps if now - t < 60]
    _search_timestamps.append(time.time())


@traceable(name="tavily_search")
def search_tavily(query: str, max_results: int = 5) -> List[dict]:
    _enforce_search_rate_limit()
    try:
        res = tavily_client.search(query=query, max_results=max_results)
        return res.get("results", []) if isinstance(res, dict) else []
    except Exception as e:
        print("Tavily search failed:", e)
        return []



# Evidence cleaning & user JSON helpers

def clean_evidence(raw: str) -> str:
    if not raw:
        return ""
    s = raw
    s = re.sub(r"```.*?```", "", s, flags=re.DOTALL)
    s = re.sub(r"\s*\\n\s*", " ", s)
    s = re.sub(r'^\s*\{.*?"evidence"\s*:\s*', "", s, flags=re.DOTALL)
    s = s.replace('"covered":', "")
    s = s.replace("{", "").replace("}", "")
    s = s.replace('"""', "").replace("'''", "")
    s = s.strip()
    s = re.sub(r'^\s*["\']?evidence["\']?\s*[:\-]?\s*', "", s, flags=re.I)
    s = html.unescape(s).strip()
    if len(s) > 300:
        s = s[:300].rsplit(" ", 1)[0] + "..."
    return s


def simplify_score_meta_for_user(score_meta: Optional[dict]):
    if not score_meta:
        return None
    covered = score_meta.get("covered_count", 0)
    total = score_meta.get("total", 1)
    objectives = []
    for d in score_meta.get("details", []):
        objectives.append(
            {
                "objective": d.get("objective"),
                "covered": True if str(d.get("covered", "no")).lower() == "yes" else False,
                "evidence": clean_evidence(d.get("evidence", "")),
            }
        )
    coverage_percent = int(round((covered / total) * 100))
    summary = f"{covered}/{total} objectives covered"
    explain = f"{coverage_percent}% — {summary}"
    return {
        "coverage_percent": coverage_percent,
        "summary": summary,
        "explain": explain,
        "objective_reports": objectives,
    }


# Embeddings + Temporary in-memory vector store (Milestone 2)

_emb_model = None
_np = None
_EMBEDDING_DEBUG = os.getenv("EMBEDDING_DEBUG", "0") in ("1", "true", "True")

# Chunking configuration
CHUNK_SIZE = 1200
CHUNK_OVERLAP = 250
MIN_CHUNK_LENGTH = 300

try:
    from sentence_transformers import SentenceTransformer
    import numpy as np

    try:
        print("[Embedding] Loading SentenceTransformer 'all-MiniLM-L6-v2' ... (this may take a few seconds)")
        _emb_model = SentenceTransformer("all-MiniLM-L6-v2")
        _np = np
        print("[Embedding] Model loaded:all-MiniLM-L6-v2")
    except Exception as e:
        print(f"[Embedding] Failed to load SentenceTransformer model: {e}")
        _emb_model = None
        _np = None
except Exception:
    _emb_model = None
    try:
        import numpy as np
        _np = np
    except Exception:
        _np = None


def is_context_relevant_semantically(
    context: str,
    cp,
    threshold: float = 0.35
) -> bool:
    """
    Embedding-based semantic relevance check.
    Returns False if context is unrelated to checkpoint topic/objectives.
    """
    if not context.strip():
        return False

    # If embeddings unavailable, don't block pipeline
    if _emb_model is None or _np is None:
        return True

    reference_text = cp.topic + " " + " ".join(cp.objectives)

    try:
        ctx_vec = _emb_model.encode([context[:2000]], convert_to_numpy=True)
        ref_vec = _emb_model.encode([reference_text], convert_to_numpy=True)

        ctx_vec = ctx_vec / (_np.linalg.norm(ctx_vec, axis=1, keepdims=True) + 1e-12)
        ref_vec = ref_vec / (_np.linalg.norm(ref_vec, axis=1, keepdims=True) + 1e-12)

        similarity = float(ctx_vec @ ref_vec.T)

        if _EMBEDDING_DEBUG:
            print(f"[Semantic Relevance] similarity={similarity:.3f}")

        return similarity >= threshold

    except Exception:
        return True




def build_temp_vector_store(chunks: List[str]):
    """
    Build temporary in-memory vector store for this session.
    Returns dict: { 'chunks': [...], 'vectors': np.array or None, 'meta': {...} }
    """
    if not chunks:
        return {"chunks": [], "vectors": None, "meta": {"embeddings_used": False}}
    if _emb_model and _np is not None:
        try:
            vecs = _emb_model.encode(chunks, convert_to_numpy=True, show_progress_bar=False)
            # normalize vectors for cosine sim
            norms = _np.linalg.norm(vecs, axis=1, keepdims=True) + 1e-12
            vecs = vecs / norms
            store = {"chunks": chunks, "vectors": vecs, "meta": {"embeddings_used": True}}
            if _EMBEDDING_DEBUG:
                print(f"[Embedding] Built vector store: vectors shape = {vecs.shape}")
            return store
        except Exception as e:
            print(f"[Embedding] Exception while encoding chunks: {e}")
            pass
    # fallback: no embeddings (vectors=None) — token-overlap will be used
    if _EMBEDDING_DEBUG:
        print("[Embedding] No embedding model available; using token-overlap fallback.")
    return {"chunks": chunks, "vectors": None, "meta": {"embeddings_used": False}}


def embedding_debug_print(store, label: str = ""):
    try:
        emb_used = bool(store and store.get("vectors") is not None)
        if emb_used:
            vecs = store["vectors"]
            shape = getattr(vecs, "shape", None)
            print(f"[Embedding Confirm] {label} embeddings used: True | vector shape: {shape}")
            # print a tiny sample: first vector first 6 values
            sample = vecs[0][:6].tolist() if hasattr(vecs[0], "tolist") else list(vecs[0][:6])
            print(f"[Embedding Confirm] sample vec[0][:6] ~ {sample}")
        else:
            print(f"[Embedding Confirm] {label} embeddings used: False")
    except Exception as e:
        print(f"[Embedding Confirm] Error during debug print: {e}")


def retrieve_top_k(store, query: str, k: int = 3) -> List[str]:
    chunks = store.get("chunks", []) or []
    vectors = store.get("vectors", None)

    if not chunks:
        return []

    if vectors is not None and _emb_model is not None and _np is not None:
        try:
            qv = _emb_model.encode([query], convert_to_numpy=True, show_progress_bar=False)
            qv = qv / (_np.linalg.norm(qv, axis=1, keepdims=True) + 1e-12)

            sims = vectors @ qv.T
            sims = sims.reshape(-1)   # ✅ CRITICAL FIX

            idx_sorted = sims.argsort()[::-1][:k]
            top_chunks = [chunks[i] for i in idx_sorted if i < len(chunks)]

            if _EMBEDDING_DEBUG:
                top_info = [(int(i), float(sims[i])) for i in idx_sorted]
                print(f"[Retrieve Debug] top-k indices+sim: {top_info}")

            return top_chunks

        except Exception as e:
            print(f"[Retrieve] Embedding retrieval failed: {e}")

    # ---- fallback: token overlap ----
    q_words = set(re.findall(r"\w+", query.lower()))
    scored = []
    for i, c in enumerate(chunks):
        c_words = set(re.findall(r"\w+", c.lower()))
        scored.append((len(q_words & c_words), i))

    scored.sort(reverse=True)
    return [chunks[i] for s, i in scored[:k] if s > 0] or chunks[:k]

    if top:
        if _EMBEDDING_DEBUG:
            print(f"[Retrieve Debug] token-overlap scores (top): {scored[:k]}")
        return top
    # last fallback: first k chunks
    return chunks[:k]



# LangGraph Nodes

def get_checkpoint_by_id(cp_id: str) -> Checkpoint:
    for cp in CHECKPOINTS:
        if cp.id == cp_id:
            return cp
    raise ValueError(f"Checkpoint {cp_id} not found")

def create_custom_checkpoint(topic: str) -> Checkpoint:
    return Checkpoint(
        id="__custom__",
        topic=topic.strip(),
        objectives=[
            f"Explain the core principles of {topic}",
            f"Describe key laws or rules related to {topic}",
            f"Apply {topic} concepts to simple examples",
        ],
        success_criteria=f"Learner can clearly explain and apply {topic} concepts."
    )
def start_checkpoint(state: AgentState) -> AgentState:
    state = dict(state)

    # ======================================================
    # 1️⃣ Resolve checkpoint (predefined OR custom)
    # ======================================================
    if state.get("cp_id") == "__custom__":
        topic = (state.get("custom_topic") or "").strip()
        if not topic:
            raise ValueError("Custom topic provided but empty.")

        # ✅ USE SINGLE SOURCE OF TRUTH
        cp = create_custom_checkpoint(topic)

        # 🔹 OPTIONAL: AI-enhanced objectives
        try:
            prompt = f"""
Generate exactly 3 beginner-friendly learning objectives for the topic:
{topic}

Rules:
- Conceptual, not mathematical
- Simple sentences
- No numbering
- No extra text

Return JSON only:
{{ "objectives": ["...", "...", "..."] }}
"""
            raw = groq_chat(prompt, use_cache=True)
            import json, re
            m = re.search(r"\{.*\}", raw, re.DOTALL)
            if m:
                data = json.loads(m.group(0))
                if isinstance(data.get("objectives"), list) and len(data["objectives"]) >= 3:
                    cp.objectives = data["objectives"][:3]
        except Exception:
            pass  # fallback stays intact

    else:
        cp = get_checkpoint_by_id(state["cp_id"])

    # ======================================================
    # 2️⃣ Core reset
    # ======================================================
    state["checkpoint"] = cp
    state["gathered_context"] = ""
    state["context_sources"] = []
    state["score_meta"] = None

    # ======================================================
    # 3️⃣ Context processing reset
    # ======================================================
    state["processed_chunks"] = []
    state["temp_vector_store"] = None

    # ======================================================
    # 4️⃣ Assessment reset
    # ======================================================
    state["questions"] = []
    state["learner_answers"] = []
    state["score_percent"] = None
    state["pass_threshold_met"] = None
    state["question_scores"] = []

    state["refetch_attempted"] = False

    # ======================================================
    # 5️⃣ 🔒 Feynman HARD RESET (CRITICAL)
    # ======================================================
    state["feynman_explanation"] = None
    state["feynman_rounds"] = 0
    state["focus_concepts"] = None
    state["failed_objectives"] = []

    return state


@traceable(name="gather_context_node")
def gather_context(state: AgentState) -> AgentState:
    cp = state["checkpoint"]
    notes = state["user_notes"]
    pdfs = state.get("user_pdfs", [])

    sources = []

    # -------------------------------------------------
    # 1️⃣ USER NOTES (COLLECT, DO NOT DOMINATE)
    # -------------------------------------------------
    user_notes = notes.strip() if notes.strip() else ""
    if user_notes:
        sources.append("user_notes")

    # -------------------------------------------------
    # 2️⃣ AUTO-GENERATED CONTEXT (PDF + WEB ALWAYS)
    # -------------------------------------------------
    auto_context = ""

    if pdfs:
        pdf_text = gather_texts_from_pdfs(pdfs)
        if pdf_text.strip():
            auto_context += pdf_text + "\n"
            sources.append("pdf_upload")

    # ALWAYS fetch web content (even if notes exist)
    query = f"{cp.topic} - " + "; ".join(cp.objectives)
    results = search_tavily(query=query)

    for item in results:
        content = item.get("content")
        if content:
            auto_context += content + "\n"

    if auto_context.strip():
        sources.append("web_search")
        auto_context = expand_text(auto_context, cp)

    # Summarize ONLY auto content if large
    if len(auto_context) > 5000:
        auto_context = summarize_text(auto_context)

    # -------------------------------------------------
    # 3️⃣ MERGE USER NOTES + GENERATED CONTENT
    # -------------------------------------------------
    if user_notes and auto_context.strip():
        merge_prompt = f"""
You are an expert tutor.

Merge the USER NOTES with the GENERATED CONTENT into ONE clear explanation.

RULES:
- Keep all correct technical information
- If user notes are correct, integrate them naturally
- If user notes are incomplete, enrich them
- If user notes are incorrect, silently correct them
- Do NOT mention notes or sources
- Write a clean instructional explanation
- Stay strictly within the topic

TOPIC:
{cp.topic}

LEARNING OBJECTIVES:
{json.dumps(cp.objectives, ensure_ascii=False)}

USER NOTES:
\"\"\"{user_notes}\"\"\"

GENERATED CONTENT:
\"\"\"{auto_context[:6000]}\"\"\"

FINAL EXPLANATION:
"""
        final_context = groq_chat(merge_prompt, use_cache=False).strip()
    else:
        # No notes → pure generated content
        final_context = auto_context

    # -------------------------------------------------
    # 4️⃣ SAVE FINAL CONTEXT
    # -------------------------------------------------
    state = dict(state)
    state["gathered_context"] = final_context
    state["context_sources"] = list(dict.fromkeys(sources))  # dedupe

    print("\n" + "=" * 80)
    print(f"[Merged Context] Source(s): {', '.join(state['context_sources']) if sources else 'None'}")
    print("-" * 80)
    print(final_context[:3000] if final_context.strip() else "[No context gathered]")
    print("=" * 80 + "\n")

    return state


@traceable(name="validate_context_node")
def validate_context(state: AgentState) -> AgentState:
    cp = state["checkpoint"]
    context = state["gathered_context"]
    refetch = state.get("refetch_attempted", False)

    #  SEMANTIC FILTER
    is_relevant = is_context_relevant_semantically(context, cp)

    if not is_relevant and not refetch:
        print("[Semantic Filter] Context is unrelated. Refetching from web...")

        query = f"{cp.topic} for beginners; " + "; ".join(cp.objectives)
        results = search_tavily(query=query)

        new_context = ""
        for item in results:
            c = item.get("content")
            if c:
                new_context += c + "\n"

        if new_context.strip():
            new_context = summarize_text(new_context)

        context = new_context
        refetch = True


    def score_ctx_objectives_only(ctx: str):
        objectives = cp.objectives
        covered = 0
        details = []

        for obj in objectives:
            prompt = f"""
You must respond with a single valid JSON object and NOTHING else.

Keys:
- "covered": either "yes" or "no"
- "evidence": one short sentence (<=30 words) quoting or paraphrasing the CONTEXT

OBJECTIVE:
\"\"\"{obj}\"\"\"\n
CONTEXT:
\"\"\"{ctx[:8000]}\"\"\"\n
Return only JSON like: {{ "covered": "yes", "evidence": "..." }}
"""
            raw = groq_chat(prompt).strip()

            cov = "no"
            evidence = ""

            try:
                import json as _json, re as _re
                m = _re.search(r"\{.*\}", raw, _re.DOTALL)
                if m:
                    parsed = _json.loads(m.group(0))
                    cov = str(parsed.get("covered", "no")).lower()
                    evidence = str(parsed.get("evidence", "")).strip()
                    if cov not in ("yes", "no"):
                        cov = "no"
            except Exception:
                pass

            if cov == "yes":
                covered += 1

            details.append({
                "objective": obj,
                "covered": cov,
                "evidence": evidence
            })

        total = len(objectives) or 1
        base_score = round((covered / total) * 5)
        score = min(5, max(1, base_score))

        meta = {
            "covered_count": covered,
            "total": total,
            "details": details
        }
        return score, meta

    score, score_meta = score_ctx_objectives_only(context)

    if score <= 2 and not refetch:
        print("Low relevance detected. Refetching...")
        query = f"{cp.topic} for beginners; " + "; ".join(cp.objectives)
        results = search_tavily(query=query)

        new_context = ""
        for item in results:
            c = item.get("content")
            if c:
                new_context += c + "\n"

        if new_context.strip():
            new_context = summarize_text(new_context)

        score, score_meta = score_ctx_objectives_only(new_context)
        context = new_context
        refetch = True

    state = dict(state)
    state["gathered_context"] = context
    state["relevance_score_model"] = score
    state["refetch_attempted"] = refetch
    state["score_meta"] = score_meta

    if refetch and "web_search" not in state["context_sources"]:
        state["context_sources"].append("web_search")

    print(f"Score for {cp.id}: {score} ({score_meta['covered_count']}/{score_meta['total']} objectives covered)")
    for d in score_meta["details"]:
        print(f" - Obj: {d['objective'][:60]}... => {d['covered']} | evidence: {clean_evidence(d['evidence'])[:140]}")

    return state



# Milestone 2: Context processing (chunking) + build temp vector store

@traceable(name="process_context_node")
def process_context(state: AgentState) -> AgentState:
    context = (state.get("gathered_context") or "").strip()
    chunks: List[str] = []

    if context:
        start = 0
        text_len = len(context)

        while start < text_len:
            end = start + CHUNK_SIZE
            chunk = context[start:end].strip()

            if len(chunk) >= MIN_CHUNK_LENGTH:
                chunks.append(chunk)

            # move forward with overlap
            start = end - CHUNK_OVERLAP
            if start < 0:
                start = 0

            # stop infinite loop
            if start >= text_len:
                break

    # Fallback safety
    if not chunks and context:
        chunks = [context]

    state = dict(state)
    state["processed_chunks"] = chunks

    # Build temporary vector store (Milestone 2 requirement)
    store = build_temp_vector_store(chunks)
    state["temp_vector_store"] = store

    # Debug confirmation
    if _EMBEDDING_DEBUG:
        print(f"[Chunking] Produced {len(chunks)} chunks "
              f"(size={CHUNK_SIZE}, overlap={CHUNK_OVERLAP})")
        embedding_debug_print(store, label=f"cp={state.get('cp_id')}")

    return state



@traceable(name="generate_questions_node")
def generate_questions(state: AgentState) -> AgentState:
    import time, random

    cp = state["checkpoint"]
    chunks = state.get("processed_chunks") or []
    store = state.get("temp_vector_store") or {
        "chunks": chunks,
        "vectors": None,
        "meta": {"embeddings_used": False},
    }

    # -------------------------------------------------
    # Attempt-aware variation
    # -------------------------------------------------
    attempt = state.get("feynman_rounds", 0)

    QUESTION_STYLES = [
        "role",
        "process",
        "why",
        "analogy",
        "application",
    ]
    style = QUESTION_STYLES[attempt % len(QUESTION_STYLES)]

    # 🔥 TRUE ENTROPY
    nonce = f"{attempt}-{int(time.time()*1000)}-{random.randint(1000,9999)}"

    # -------------------------------------------------
    # Weak concepts
    # -------------------------------------------------
    focus_concepts = state.get("focus_concepts") or cp.objectives
    query = " ".join(focus_concepts)

    # -------------------------------------------------
    # Retrieve top-k chunks
    # -------------------------------------------------
    focus_for_generation = ""
    if chunks:
        top_chunks = retrieve_top_k(store, query=query, k=3)
        focus_for_generation = "\n\n".join(top_chunks)

    base_context = (focus_for_generation or "\n\n".join(chunks))[:3000]

    # -------------------------------------------------
    # LLM prompt (18 WORD LIMIT)
    # -------------------------------------------------
    prompt = f"""
You are an adaptive AI tutor.

SYSTEM NONCE: {nonce}

ATTEMPT NUMBER: {attempt}
QUESTION STYLE: {style}

Generate EXACTLY 3 SHORT questions.

STRICT FORMAT RULES:
- ONE sentence only
- MAXIMUM 18 words ⬅️ UPDATED
- NO commas
- NO chained clauses
- MUST end with ?
- SIMPLE beginner language
- Focus ONLY on weak concepts
- DIFFERENT from previous attempts
- DO NOT start with: "What is", "Define", "Explain"

WEAK CONCEPTS:
{json.dumps(focus_concepts, ensure_ascii=False)}

Return ONLY valid JSON:
{{ "questions": ["Q1", "Q2", "Q3"] }}
"""

    raw = groq_chat(prompt, use_cache=False).strip()

    questions = []
    try:
        m = re.search(r"\{.*\}", raw, re.DOTALL)
        if m:
            data = json.loads(m.group(0))
            qlist = data.get("questions", [])
            if isinstance(qlist, list):
                questions = [q.strip() for q in qlist if isinstance(q, str)]
    except Exception:
        pass

    # -------------------------------------------------
    # Final safety trimming (18 words)
    # -------------------------------------------------
    clean_questions = []
    for q in questions:
        q = re.sub(r",.*", "", q)
        q = " ".join(q.split()[:18])        # ⬅️ UPDATED
        if not q.endswith("?"):
            q += "?"
        clean_questions.append(q)

    state = dict(state)
    state["questions"] = clean_questions[:3]

    print(f"[Question Generation] attempt={attempt}, style={style}, nonce={nonce}")
    for q in state["questions"]:
        print(" -", q)

    return state

def keyword_overlap_ratio(answer: str, context: str) -> float:
    if not answer or not context:
        return 0.0

    a_words = set(re.findall(r"\b[a-zA-Z]{4,}\b", answer.lower()))
    c_words = set(re.findall(r"\b[a-zA-Z]{4,}\b", context.lower()))

    if not a_words:
        return 0.0

    return len(a_words & c_words) / len(a_words)

def verify_understanding(state: AgentState) -> AgentState:

    cp = state["checkpoint"]
    questions = state.get("questions") or []
    answers = state.get("learner_answers") or []

    store = state.get("temp_vector_store") or {
        "chunks": state.get("processed_chunks", []),
        "vectors": None,
        "meta": {"embeddings_used": False},
    }

    if not questions or not answers:
        print(f"No questions or learner answers for {cp.id}; skipping verification.")
        state["score_percent"] = None
        state["pass_threshold_met"] = None
        return state

    n = min(len(questions), len(answers))
    if n == 0:
        state["score_percent"] = None
        state["pass_threshold_met"] = None
        return state

    scores = []
    question_scores = []
    failed_objectives = set()

    for i in range(n):
        q = questions[i]
        a = (answers[i] or "").strip()

        # -------------------------------------------------
        # 1️⃣ SHORT ANSWER CHECK
        # -------------------------------------------------
        if len(a) < 30:
            score_val = 0
            scores.append(score_val)
            question_scores.append({
                "question": q,
                "answer": a,
                "score": score_val,
            })
            failed_objectives.add(q)
            print(f"[Verify] {cp.id} Q{i+1} → score 0 (answer too short)")
            continue

        # -------------------------------------------------
        # CONTEXT FOR GRADING
        # -------------------------------------------------
        top_chunks = retrieve_top_k(store, query=q, k=3)
        context_for_grading = "\n\n".join(top_chunks)[:4000]

        # -------------------------------------------------
        # 2️⃣ KEYWORD OVERLAP GUARD
        # -------------------------------------------------
        overlap = keyword_overlap_ratio(a, context_for_grading)

        if overlap < 0.05:
            score_val = 0
            scores.append(score_val)
            question_scores.append({
                "question": q,
                "answer": a,
                "score": score_val,
            })
            failed_objectives.add(q)
            print(f"[Verify] {cp.id} Q{i+1} → score 0 (unrelated answer)")
            continue

        # -------------------------------------------------
        # 3️⃣ DETERMINISTIC BUCKETED SCORING
        # -------------------------------------------------
        prompt = f"""
Return ONLY valid JSON.

CONTEXT:
\"\"\"{context_for_grading}\"\"\"

QUESTION:
{q}

LEARNER ANSWER:
{a}

Choose EXACTLY ONE score from:
[0, 40, 70, 100]

Scoring rules:
- 0 → wrong or unrelated
- 40 → partial understanding
- 70 → correct with minor gaps
- 100 → fully correct

Return:
{{ "score": number }}
"""

        score_val = cached_grade(prompt)

        if score_val not in (0, 40, 70, 100):
            score_val = 0

        scores.append(score_val)
        question_scores.append({
            "question": q,
            "answer": a,
            "score": score_val,
        })

        # -------------------------------------------------
        # 4️⃣ FAILURE TRACKING
        # -------------------------------------------------
        if score_val < 70:
            failed_objectives.add(q)

        print(f"[Verify] {cp.id} Q{i+1} score: {score_val}")

    # -------------------------------------------------
    # 5️⃣ STRICT PASS LOGIC
    # -------------------------------------------------
    passed = all(s >= 70 for s in scores)

    avg_score = sum(scores) / len(scores)

    state["score_percent"] = avg_score
    state["pass_threshold_met"] = passed
    state["question_scores"] = question_scores
    state["failed_objectives"] = list(failed_objectives)

    print(f"\nOverall quiz score for {cp.id}: {avg_score:.1f}% | passed={passed}")

    return state



@traceable(name="feynman_node")
def feynman_node(state: AgentState) -> AgentState:

    cp = state["checkpoint"]
    weak_concepts = state.get("failed_objectives") or cp.objectives

    store = state.get("temp_vector_store") or {
        "chunks": state.get("processed_chunks", []),
        "vectors": None,
    }

    supporting_chunks = retrieve_top_k(
        store,
        query=" ".join(weak_concepts),
        k=3
    )

    support_context = "\n\n".join(supporting_chunks)[:2500]

    prompt = f"""
You are teaching using the Feynman Technique.

TOPIC:
{cp.topic}

FAILED CONCEPTS:
{json.dumps(weak_concepts, ensure_ascii=False)}

TRUSTED CONTEXT:
\"\"\"{support_context}\"\"\"

Rules:
- Explain ONLY the failed concepts
- Stay strictly within the topic
- Use simple language
- Use ONE real-life analogy per concept
- Step-by-step
- Assume zero prior knowledge
- Do NOT introduce new topics
- Make this explanation DIFFERENT from previous attempts
"""

    explanation = groq_chat(prompt, use_cache=False).strip()

    state = dict(state)



    state["feynman_explanation"] = explanation
    state["focus_concepts"] = weak_concepts
    state["feynman_rounds"] += 1

    # Reset assessment state


    state["learner_answers"] = []
    state["score_percent"] = None

    print("\n" + "=" * 80)
    print("🧠 FEYNMAN TEACHING MODE")
    print("Topic:", cp.topic)
    print("Weak concepts:", weak_concepts)
    print("=" * 80)
    print(explanation)
    print("=" * 80)

    return state



def route_after_verification(state: AgentState) -> str:
    passed = state.get("pass_threshold_met", False)
    attempts = state.get("feynman_rounds", 0)

    if passed:
        return "pass"

    if attempts < 2:
        return "feynman"

    return "pass"


def run_checkpoint_with_answers(cp_id: str, answers: List[str], prev_state=None):
    """
    Runs ONE checkpoint with provided answers.
    Used by Streamlit UI.
    """

    graph = build_graph()

    if prev_state is None:
        state: AgentState = {
            "cp_id": cp_id,
            "checkpoint": None,
            "user_notes": "",
            "user_pdfs": [],
            "gathered_context": "",
            "context_sources": [],
            "relevance_score_model": None,
            "refetch_attempted": False,
            "score_meta": None,
            "processed_chunks": [],
            "questions": [],
            "learner_answers": [],
            "score_percent": None,
            "pass_threshold_met": None,
            "temp_vector_store": None,
            "feynman_explanation": None,
            "feynman_rounds": 0,
            "focus_concepts": None,
        }
    else:
        state = prev_state

    # Inject learner answers
    state["learner_answers"] = answers

    final_state = graph.invoke(state)
    return final_state




def build_graph():
    g = StateGraph(AgentState)

    g.add_node("start_checkpoint", start_checkpoint)
    g.add_node("gather_context", gather_context)
    g.add_node("validate_context", validate_context)
    g.add_node("process_context", process_context)
    g.add_node("generate_questions", generate_questions)
    g.add_node("verify_understanding", verify_understanding)
    g.add_node("feynman_node", feynman_node)

    g.set_entry_point("start_checkpoint")

    g.add_edge("start_checkpoint", "gather_context")
    g.add_edge("gather_context", "validate_context")
    g.add_edge("validate_context", "process_context")
    g.add_edge("process_context", "generate_questions")
    g.add_edge("generate_questions", "verify_understanding")

    # 🔁 Decision point
    g.add_conditional_edges(
        "verify_understanding",
        route_after_verification,
        {
            "pass": END,
            "feynman": "feynman_node",
        },
    )

    # 🔁 Loop back after teaching
    g.add_edge("feynman_node", "generate_questions")

    return g.compile()



graph = build_graph()



# Helper: read multi-line input (end with an 'END' line)

def read_multiline(prompt_msg: str) -> str:
    print(prompt_msg)
    print("Enter/Paste your text. End with a single line containing only: END")
    lines = []
    while True:
        try:
            line = input()
        except EOFError:
            break
        if line.strip() == "END":
            break
        lines.append(line)
    return "\n".join(lines).strip()


def all_objectives_mastered(state: AgentState) -> bool:
    """
    Returns True only if ALL objectives are marked as covered=yes
    """
    meta = state.get("score_meta")
    if not meta:
        return False

    return all(
        item.get("covered") == "yes"
        for item in meta.get("details", [])
    )

def run_single_checkpoint_interactive(cp_id: str) -> dict:
    cp = get_checkpoint_by_id(cp_id)
    print(f"\n--- Checkpoint {cp.id}: {cp.topic} ---")

    notes = read_multiline("Provide user notes for this checkpoint (or leave blank and type END):")
    pdfs_input = input("Enter comma-separated PDF paths for this checkpoint (or leave blank): ").strip()
    pdfs = [p.strip() for p in pdfs_input.split(",") if p.strip()]

    # ---------------------------
    # INITIAL STATE
    # ---------------------------
    state: AgentState = {
        "cp_id": cp_id,
        "checkpoint": None,
        "user_notes": notes,
        "user_pdfs": pdfs,
        "gathered_context": "",
        "context_sources": [],
        "relevance_score_model": None,
        "refetch_attempted": False,
        "score_meta": None,
        "processed_chunks": [],
        "questions": [],
        "learner_answers": [],
        "score_percent": None,
        "pass_threshold_met": None,
        "temp_vector_store": None,
        "feynman_explanation": None,
        "feynman_rounds": 0,
        "focus_concepts": None,
    }

    # ---------------------------
    # INITIAL PIPELINE (RUN ONCE)
    # ---------------------------
    state = start_checkpoint(state)
    state = gather_context(state)
    state = validate_context(state)
    state = process_context(state)

    MAX_REASSESSMENTS = 3

    # ===========================
    # 🔁 TEACH–ASSESS LOOP
    # ===========================
    while True:

        # 1️⃣ Generate questions
        state = generate_questions(state)

        print("\nGenerated Questions:")
        for i, q in enumerate(state["questions"], 1):
            print(f"{i}. {q}")

        # 2️⃣ Collect answers
        answers = []
        for i, q in enumerate(state["questions"], 1):
            ans = read_multiline(f"\nAnswer for Q{i}: {q}")
            answers.append(ans)

        state["learner_answers"] = answers

        # 3️⃣ Verify understanding
        state = verify_understanding(state)

        # 4️⃣ STRICT EXIT CONDITION
        if state.get("pass_threshold_met") and all_objectives_mastered(state):
            print("\n✅ Passed! All concepts mastered.")
            break

        # 5️⃣ Stop if max reteaching reached
        if state.get("feynman_rounds", 0) >= MAX_REASSESSMENTS:
            print("\n⛔ Maximum re-teaching attempts reached.")
            break

        # 6️⃣ Feynman remediation
        print("\n❌ Score below threshold — entering Feynman teaching mode\n")
        state = feynman_node(state)

        # 7️⃣ Re-process context so explanation affects next questions
        state = process_context(state)

        print("\n🔁 Re-attempting questions...\n")

    # ---------------------------
    # FINAL RESULT
    # ---------------------------
    return {
        "cp_id": cp_id,
        "topic": cp.topic,
        "context_score": state.get("relevance_score_model"),
        "quiz_score": state.get("score_percent"),
        "passed": state.get("pass_threshold_met") and all_objectives_mastered(state),
        "sources": state.get("context_sources"),
        "questions": state.get("questions"),
        "answers": state.get("learner_answers"),
    }


# Evaluation suite (automated tests for Q relevance & scoring)

def _generate_good_answer_for_question(context: str, question: str) -> str:

    prompt = f"""
Use the CONTEXT below to write a concise, correct, and focused answer to the QUESTION.
Answer must be at least 25 words (to avoid short-answer penalties) and at most 80 words.
Keep it factual and directly relevant to the question.

CONTEXT:
\"\"\"{context[:3500]}\"\"\"\n

QUESTION:
{question}

Answer:
"""
    raw = groq_chat(prompt)
    txt = raw.strip()
    words = re.findall(r"\w+", txt)
    if len([w for w in words if len(w) > 2]) < 15:
        fallback = f"Provide a direct, explanatory answer (>=25 words) to: {question}\nUsing: {context[:800]}"
        raw2 = groq_chat(fallback)
        txt = raw2.strip() or txt
    return txt


def run_evaluation_suite():
    """
    Runs automated evaluation across all CHECKPOINTS.
    For each checkpoint:
      - gather/process/generate questions
      - create 'good' answers via LLM (ensured to be long enough)
      - create 'bad' answers (short/off-topic)
      - run verify_understanding for both sets and record metrics
    """
    overall = []
    print("Running automated evaluation suite for all checkpoints...\n")
    for cp in CHECKPOINTS:
        # initialize state
        state: AgentState = {
            "cp_id": cp.id,
            "checkpoint": None,
            "user_notes": "",  # no notes; will use web search fallback
            "user_pdfs": [],
            "gathered_context": "",
            "context_sources": [],
            "relevance_score_model": None,
            "refetch_attempted": False,
            "score_meta": None,
            "processed_chunks": [],
            "questions": [],
            "learner_answers": [],
            "score_percent": None,
            "pass_threshold_met": None,
            "temp_vector_store": None,
        }

        state = start_checkpoint(state)
        state = gather_context(state)
        state = validate_context(state)
        state = process_context(state)
        state = generate_questions(state)

        questions = state.get("questions") or []
        context_text = state.get("gathered_context") or "\n".join(state.get("processed_chunks", [])) or ""
        store = state.get("temp_vector_store") or {"chunks": [], "vectors": None, "meta": {"embeddings_used": False}}

        # Double-confirm embedding usage for this checkpoint
        emb_used = bool(store.get("vectors") is not None)
        print(f"[Eval] Checkpoint {cp.id} embeddings_used = {emb_used}")
        if emb_used:
            try:
                embedding_debug_print(store, label=f"eval-cp={cp.id}")
            except Exception:
                pass

        # Build 'good' answers using the LLM (ensured to be long enough)
        good_answers = []
        for q in questions:
            ans = _generate_good_answer_for_question(context_text, q)
            if len(re.findall(r"\w+", ans)) < 30:
                ans = ans + " " + ("This answer expands on the main points to ensure full coverage of the objective. " * 2)
            good_answers.append(ans)

        # Build 'bad' answers (short/off-topic)
        bad_answers = ["I don't know." for _ in questions]

        # Evaluate good answers
        state_good = dict(state)
        state_good["learner_answers"] = good_answers
        state_good = verify_understanding(state_good)
        good_score = state_good.get("score_percent") or 0.0
        good_pass = state_good.get("pass_threshold_met") or False

        # Evaluate bad answers
        state_bad = dict(state)
        state_bad["learner_answers"] = bad_answers
        state_bad = verify_understanding(state_bad)
        bad_score = state_bad.get("score_percent") or 0.0
        bad_pass = state_bad.get("pass_threshold_met") or False

        q_rel = 1.0 if questions else 0.0

        overall.append({
            "cp_id": cp.id,
            "topic": cp.topic,
            "num_questions": len(questions),
            "q_rel": q_rel,
            "context_score": state.get("relevance_score_model"),
            "good_score": good_score,
            "good_pass": good_pass,
            "bad_score": bad_score,
            "bad_pass": bad_pass,
            "embeddings_used": emb_used,
        })

    # Print summary
    print("\n=== Evaluation Summary ===\n")
    total_q_rel = 0.0
    good_pass_count = 0
    bad_fail_count = 0
    emb_used_count = 0
    for r in overall:
        print(f"Checkpoint: {r['cp_id']} - {r['topic']}")
        print(f"  Questions: {r['num_questions']}, Q-rel: {r['q_rel']:.2f}")
        print(f"  Context score (1-5): {r['context_score']}")
        print(f"  Good answers -> score: {r['good_score']:.1f}%, pass: {r['good_pass']}")
        print(f"  Bad answers  -> score: {r['bad_score']:.1f}%, pass: {r['bad_pass']}")
        print(f"  Embeddings used: {r.get('embeddings_used')}")
        print()
        total_q_rel += r['q_rel']
        if r['good_pass']:
            good_pass_count += 1
        if not r['bad_pass']:
            bad_fail_count += 1
        if r.get('embeddings_used'):
            emb_used_count += 1

    n = len(overall) or 1
    print("--- Overall Metrics ---")
    print(f"Average question relevance (fraction): {total_q_rel / n:.3f}")
    print(f"Good answers pass-rate (should be high): {good_pass_count / n * 100:.1f}%")
    print(f"Bad answers fail-rate (should be high): {bad_fail_count / n * 100:.1f}%")
    print(f"Embeddings used in checkpoints: {emb_used_count}/{n}\n")

    return overall



# Interactive run for multiple checkpoints

def interactive_run():
    print("Interactive Milestone 2 runner.")
    print("Available checkpoints:")
    for cp in CHECKPOINTS:
        print(f" - {cp.id}: {cp.topic}")

    chosen = input("Enter comma-separated checkpoint ids to run (or 'all'): ").strip()
    if chosen.lower() == "all" or not chosen:
        ids = [cp.id for cp in CHECKPOINTS]
    else:
        ids = [c.strip() for c in chosen.split(",") if c.strip()]

    results = []
    for cp_id in ids:
        try:
            res = run_single_checkpoint_interactive(cp_id)
            results.append(res)
        except Exception as e:
            print(f"Error running {cp_id}: {e}")

    # Build markdown summary table
    lines = []
    lines.append("### Summary Table\n")
    lines.append("| Topic | Objectives | Sources | Context Score (1–5) | Quiz Score (%) | Pass (>=70%) |")
    lines.append("|-------|------------|---------|---------------------|----------------|--------------|")
    for r in results:
        cp = get_checkpoint_by_id(r["cp_id"])
        obj_list = [f"- {o}" for o in cp.objectives]
        objectives_str = "<br>".join(obj_list)
        sources_str = ", ".join(r["sources"]) if r["sources"] else "None"
        context_score = r["context_score"]
        quiz_score = r["quiz_score"]
        quiz_display = "-" if quiz_score is None else f"{quiz_score:.1f}"
        passed = r["passed"]
        passed_str = "✅" if passed else ("❌" if passed is not None else "-")
        lines.append(f"| {r['topic']} | {objectives_str} | {sources_str} | {context_score} | {quiz_display} | {passed_str} |")

    table_md = "\n".join(lines)
    try:
        display(Markdown(table_md))
    except Exception:
        print(table_md)

    # Save ALL raw LLM prompt+response logs into ONE PDF
    save_all_raw_to_one_pdf("all_llm_raw_output.pdf")


# Main

if __name__ == "__main__":
    print("Milestone 1 & 2 runner")
    print("Options:\n  1) interactive run (generate questions & answer interactively)\n  2) run evaluation suite (automated tests for Q relevance & scoring)")
    choice = input("Enter 1 or 2 (default 1): ").strip() or "1"
    if choice == "2":
        run_evaluation_suite()
    else:
        interactive_run()


Overwriting backend.py


In [ ]:
%%writefile app.py

In [29]:
%%writefile app.py

import streamlit as st
from backend import (
    CHECKPOINTS,
    get_checkpoint_by_id,
    start_checkpoint,
    gather_context,
    validate_context,
    process_context,
    generate_questions,
    verify_understanding,
    feynman_node,
)

# ======================================================
# CONFIG
# ======================================================
st.set_page_config(page_title="Adaptive AI Tutor", layout="wide")

MAX_FEYNMAN_ROUNDS = 3  # maximum attempts

# ======================================================
# GLOBAL STYLES
# ======================================================
st.markdown("""
<style>
html, body {
    background-color: #0b0f19;
    color: #e5e7eb;
    font-family: Inter, system-ui, sans-serif;
}
.card {
    background: rgba(255,255,255,0.05);
    padding: 1.5rem;
    border-radius: 14px;
    margin-bottom: 1.2rem;
}
.good { box-shadow: 0 0 25px rgba(34,197,94,0.85); }
.bad  { box-shadow: 0 0 25px rgba(239,68,68,0.85); }
.title { font-size: 2rem; font-weight: 700; }
.subtitle { color: #9ca3af; }
.score-big { font-size: 3rem; font-weight: 800; }
.attempts {
    font-size: 0.95rem;
    color: #fbbf24;
    margin-top: 0.5rem;
}
</style>
""", unsafe_allow_html=True)

# ======================================================
# SESSION STATE
# ======================================================
if "phase" not in st.session_state:
    st.session_state.phase = "context"

if "state" not in st.session_state:
    st.session_state.state = None

if "questions" not in st.session_state:
    st.session_state.questions = []

if "answers" not in st.session_state:
    st.session_state.answers = []

if "attempt_id" not in st.session_state:
    st.session_state.attempt_id = 0

# ======================================================
# SIDEBAR
# ======================================================
st.sidebar.markdown("## 🧭 Learning Path")

checkpoint_options = ["__custom__"] + [c.id for c in CHECKPOINTS]

cp_id = st.sidebar.selectbox(
    "Checkpoint",
    checkpoint_options,
    format_func=lambda x: (
        "➕ Custom Topic"
        if x == "__custom__"
        else f"{x} — {get_checkpoint_by_id(x).topic}"
    ),
)


if st.sidebar.button("🔄 Reset Session"):
    st.session_state.clear()
    st.rerun()

cp = None
if cp_id != "__custom__":
    cp = get_checkpoint_by_id(cp_id)




custom_topic = None
if cp_id == "__custom__":
    custom_topic = st.sidebar.text_input(
        "Enter topic name",
        placeholder="e.g. Newton's Laws of Motion",
    )
# ======================================================
# HEADER
# ======================================================
st.markdown(f"""
<div class="card">
  <div class="title">🧠 Adaptive AI Tutor</div>
  <div class="subtitle">
    {custom_topic if (cp_id == "__custom__" and custom_topic) else (cp.topic if cp else "")}

</div>

</div>
""", unsafe_allow_html=True)

# ======================================================
# OBJECTIVES
# ======================================================
with st.expander("🎯 Learning Objectives", expanded=True):
    st.markdown("<div class='card'>", unsafe_allow_html=True)

    if cp_id != "__custom__":
        for o in cp.objectives:
            st.markdown(f"- {o}")
    else:
        generated_cp = st.session_state.state.get("checkpoint") if st.session_state.state else None
        if generated_cp and getattr(generated_cp, "objectives", None):
            for o in generated_cp.objectives:
                st.markdown(f"- {o}")
        else:
            st.markdown("- Objectives will be generated automatically")

    st.markdown("</div>", unsafe_allow_html=True)


# ======================================================
# PHASE 1 — CONTEXT
# ======================================================
if st.session_state.phase == "context":

    notes = st.text_area("Your notes (optional)", height=160)

    load_disabled = (cp_id == "__custom__" and not custom_topic)
    if cp_id == "__custom__" and not custom_topic:
      st.warning("⚠️ Please enter a topic name to load content")

    if st.button("📚 Load Content", disabled=load_disabled):

        state = {
            "cp_id": cp_id,
            "custom_topic": custom_topic,
            "checkpoint": None,
            "user_notes": notes,
            "user_pdfs": [],
            "gathered_context": "",
            "context_sources": [],
            "processed_chunks": [],
            "questions": [],
            "learner_answers": [],
            "score_percent": None,
            "pass_threshold_met": None,
            "question_scores": [],
            "temp_vector_store": None,
            "feynman_explanation": None,
            "feynman_rounds": 0,
            "focus_concepts": None,
            "failed_objectives": [],
            "refetch_attempted": False,
        }

        state = start_checkpoint(state)
        state = gather_context(state)
        state = validate_context(state)
        state = process_context(state)

        st.session_state.state = state
        st.session_state.attempt_id = 0
        st.rerun()

    if st.session_state.state and st.session_state.state.get("gathered_context"):
        st.markdown("<div class='card'>", unsafe_allow_html=True)
        st.text_area(
            "Learning Content",
            st.session_state.state["gathered_context"],
            height=380,
        )
        st.markdown("</div>", unsafe_allow_html=True)

        if st.button("📝 Take Test"):
            s = generate_questions(st.session_state.state)
            st.session_state.state = s
            st.session_state.questions = s["questions"]
            st.session_state.answers = [""] * len(s["questions"])
            st.session_state.phase = "test"
            st.rerun()

# ======================================================
# PHASE 2 — TEST
# ======================================================
if st.session_state.phase == "test":

    s = st.session_state.state
    attempts_left = max(0, MAX_FEYNMAN_ROUNDS - s.get("feynman_rounds", 0))  # ⭐ NEW

    st.markdown(
        f"<div class='card attempts'>🔁 Attempts left: <b>{attempts_left}</b></div>",
        unsafe_allow_html=True
    )

    for i, q in enumerate(st.session_state.questions):
        st.markdown(
            f"<div class='card'>❓ <b>Q{i+1}</b>: {q}</div>",
            unsafe_allow_html=True
        )
        st.session_state.answers[i] = st.text_area(
            "",
            key=f"a{st.session_state.attempt_id}_{i}",
            height=120,
        )

    if st.button("✅ Submit Answers"):
        state = dict(st.session_state.state)
        state["learner_answers"] = st.session_state.answers

        state = verify_understanding(state)
        if not state.get("pass_threshold_met"):
          state["last_attempt_score"] = state.get("score_percent")
        if (
            not state.get("pass_threshold_met")
            and state.get("feynman_rounds", 0) < MAX_FEYNMAN_ROUNDS
        ):
            state = feynman_node(state)

        st.session_state.state = state
        st.session_state.phase = "result"
        st.rerun()

# ======================================================
# PHASE 3 — RESULT
# ======================================================
if st.session_state.phase == "result":

    s = st.session_state.state
    overall = (
    s.get("score_percent")
    if s.get("score_percent") is not None
    else s.get("last_attempt_score", 0)
)

    q_scores = s.get("question_scores") or []
    attempts_left = max(0, MAX_FEYNMAN_ROUNDS - s.get("feynman_rounds", 0))  # ⭐ NEW

    passed = bool(s.get("pass_threshold_met"))
    glow = "good" if passed else "bad"
    emoji = "🎉😄" if passed else "😕❌"

    st.markdown(f"""
    <div class="card {glow}">
        <div class="score-big">{overall:.1f}%</div>
        <div>{emoji} Overall Performance</div>
        <div class="attempts">🔁 Attempts left: <b>{attempts_left}</b></div>
    </div>
    """, unsafe_allow_html=True)

    st.markdown("### 📊 Question-wise Feedback")
    for i, qs in enumerate(q_scores, 1):
        score = qs["score"]
        st.markdown(
            f"<div class='card'><b>Q{i}:</b> {score}% {'✅' if score >= 70 else '❌'}</div>",
            unsafe_allow_html=True
        )

    if not passed and attempts_left > 0:
        st.markdown("### 🧠 Simplified Explanation")
        st.info(s.get("feynman_explanation", "Explanation unavailable."))

        if st.button("🔁 Try Again"):
            st.session_state.attempt_id += 1

            retry_state = dict(s)

            # 🔥 REQUIRED RESET
            retry_state["feynman_explanation"] = None



            retry_state = generate_questions(retry_state)

            st.session_state.state = retry_state
            st.session_state.questions = retry_state["questions"]
            st.session_state.answers = [""] * len(retry_state["questions"])
            st.session_state.phase = "test"
            st.rerun()


    elif not passed:
        st.error("⛔ No attempts left. Please reset or move on.")

    else:
        st.balloons()
        st.success("🎉 Mastered! You passed this checkpoint.")


Overwriting app.py


In [30]:
!nohup streamlit run app.py \
  --server.port 8501 \
  --server.address 0.0.0.0 \
  > /content/streamlit.log 2>&1 &


In [33]:
!lsof -i :8501


COMMAND   PID USER   FD   TYPE DEVICE SIZE/OFF NODE NAME
streamlit 484 root    6u  IPv4  29171      0t0  TCP *:8501 (LISTEN)


In [32]:
from pyngrok import ngrok

ngrok.kill()


In [34]:
from pyngrok import ngrok

public_url = ngrok.connect(8501)
print("🌍 Streamlit Public URL:", public_url)

🌍 Streamlit Public URL: NgrokTunnel: "https://unflushed-hailee-unaccorded.ngrok-free.dev" -> "http://localhost:8501"
